In [ ]:
library("httr")
library("jsonlite")
library("ggplot2")
library('RJSplot')
library('dplyr')

In [ ]:
base <- "193.175.238.88:8080/api/politicians/"
call1 <- paste(base,"all/", sep="")
get_politicians_df <- as.data.frame(fromJSON(content(GET(call1), "text", encoding="UTF-8"), flatten = TRUE))
# In the table 'get_politicians_df' you can see all the extracted politicians with respected twitter, facebook and wikipedia ids.

tables_rjs(get_politicians_df, jupyter = TRUE)

In [ ]:
# Please choose a party from the table above and input the `party` and `aggregation` variables below:
party <- "CDU"
aggregation <- "month" # aggregation can be by day ('day'), week ('week') or month('month')

party_df <- get_politicians_df[grepl(party, get_politicians_df$affiliation, fixed=TRUE) & !is.null(get_politicians_df$tw_ids) ,]
tweets <- data.frame()
replies <- data.frame()
for (tw_id in party_df$tw_ids) {
    for (tw in tw_id) {
        call_tweets <- paste(base,"twitter/tweets_by/politicians/user_id/", tw, "/?aggregate_by=", aggregation, sep="")
        call_replies <- paste(base,"twitter/replies_to/politicians/user_id/", tw, "/?aggregate_by=", aggregation, sep="")
        if (length(fromJSON(content(GET(call_tweets), "text", encoding="UTF-8"), flatten = TRUE)$values) != 0)  {
            tweets_unique <- as.data.frame(fromJSON(content(GET(call_tweets), "text", encoding="UTF-8"), flatten = TRUE))
            tweets_unique$twitter_user_id = tw
            tweets <- rbind(tweets, tweets_unique)
            }
        if (length(fromJSON(content(GET(call_replies), "text", encoding="UTF-8"), flatten = TRUE)$values) != 0)  {
            replies_unique <- as.data.frame(fromJSON(content(GET(call_replies), "text", encoding="UTF-8"), flatten = TRUE))
            replies_unique$twitter_user_id = tw
            replies <- rbind(replies, replies_unique)
            }
    }
}
tweets <- summarise(group_by(tweets, labels), tweets = sum(values), response_type = aggregation)
replies <- summarise(group_by(replies, labels), replies = sum(values), response_type = aggregation)
merged <- merge(tweets, replies, by='labels')

In [ ]:
#plotting
options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = merged, mapping = aes(labels)) +
  geom_line(aes(y = tweets, color="Tweets", group=response_type.x)) +
  geom_line(aes(y = replies, color="Replies", group=response_type.y)) +
  labs(title = "The number of aggregated tweets for an affiliated party", x = "Date", y = "N") +
  theme(axis.text.x = element_text( size = 7, angle = 60, hjust = 0.5, vjust = 0.5),
        axis.text.y = element_text( size = 7), legend.title = element_blank(), plot.title = element_text(size=10))